# Hookを使ってみる
ここでは、Transformersのモデルの、特定の層の出力を hook を使って取得してみる。  
関数は get_hidden_state_with_hook.py に定義しているので、これを使って、目的のものが hook で取得できるか確認する。



### 準備

In [1]:
# ライブラリのインポート
from pprint import pprint

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# ローカルの関数をインポート
from get_hidden_stete_with_hook import main as get_hidden_state

In [2]:
model_name_or_path = "gpt2"
prompt = "Tokyo is the capital of Japan."

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

### Hookを使って隠れ状態を取得する


In [3]:
hook_layer_index = 0
hook_result = get_hidden_state(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    positional_args_keys=None,  # 入力は保持しない
    output_keys=["hidden_states"],
    layer_index=hook_layer_index,
)
pprint(hook_result)

BatchHiddenStateObservationResult:
	hidden_states: torch.Size([1, 8, 768])



### Transformersライブラリが提供する中間表現を取得する

In [4]:
inputs = tokenizer(prompt, return_tensors="pt")
result = model.generate(
    **inputs,
    pad_token_id=tokenizer.eos_token_id,
    max_new_tokens=1,
    do_sample=False,
    output_hidden_states=True,
    return_dict_in_generate=True,
)
pprint(result.keys())

odict_keys(['sequences', 'hidden_states', 'past_key_values'])


### Hookで取得した中間表現とTransformersライブラリの出力を比較する

In [5]:
# フックで取得した出力と、Transformersライブラリが提供している中間表現を比較する
torch.allclose(
    result.hidden_states[0][1],  # 1番目の隠れ状態は最初の層の出力に対応
    hook_result.hidden_states[
        0
    ],  # hook_result["output"] はフックで取得した隠れ状態を含む
)

True